In [2]:
# importation of depedencies

import pandas as pd 
import numpy as np



## Loading data

In [6]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv("data/test.csv")



In [ ]:
data_test.isna().sum()
# We can see that there's a missing data nan


id               0
day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    1
windspeed        0
dtype: int64

In [5]:
data_test["winddirection"].mean()

np.float64(103.92318244170096)

In [6]:
data_test["winddirection"] = data_test["winddirection"].fillna(data_test["winddirection"].mean())
# Not correct way but it works

In [7]:
data_test.isna().sum()

id               0
day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
dtype: int64

### Normalisation

In [8]:
#Non-features columns ; NF_columns  -> ['id','day',"rainfall"]
features=data_train.drop(columns=['id','day','rainfall'])
#Target ; ["rainfall"]
targets=data_train["rainfall"]

x=features.values
means=x.mean(axis=0) #mean for each columns
std=x.std(axis=0)
x_normalize= (x-means)/std

In [9]:
data_train_n=data_train.copy()
data_train_n[features.columns]=x_normalize
data_train_n

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,0.671702,-0.913809,-0.642199,-0.448815,-0.199457,0.636434,0.681269,-0.729397,-0.560901,-0.465291,1
1,1,2,1.043116,-1.798289,-1.350846,-1.259418,-0.956001,1.662224,0.847728,-1.032804,-0.685925,0.009629,1
2,2,3,1.856688,-1.232222,-1.504067,-1.496667,-2.109731,-0.902250,-1.593680,1.256536,-0.435876,-0.374349,1
3,3,4,-0.035752,-1.462187,-1.178472,-1.041939,-0.691210,1.662224,1.069675,-1.032804,-0.560901,1.393971,1
4,4,5,1.449902,-0.896120,-1.063556,-1.378043,-2.052990,-3.851394,-1.704654,-0.039837,-0.810950,0.302665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2185,361,0.176484,-0.560017,-0.642199,-0.606982,-0.104888,1.918671,0.681269,-1.005222,-0.810950,0.029839,1
2186,2186,362,-0.212616,-1.621393,-1.274235,-1.160564,-0.974914,1.149329,0.681269,-1.032804,-0.685925,1.363657,1
2187,2187,363,-0.053439,-1.302980,-1.465761,-1.555980,-1.485582,-0.389355,0.181890,0.346317,-0.810950,1.121145,1
2188,2188,364,1.538334,-1.762910,-1.676440,-1.654834,-1.088396,1.277553,0.958701,-1.005222,-0.810950,-0.384453,1


#### test norm

In [11]:
X=data_test.drop(columns=['id','day'])
f=X.values
m,s=f.mean(axis=0),f.std(axis=0)
f_normalise=(X-m)/s
data_test_n=data_test.copy()
data_test_n[X.columns]=f_normalise
data_test_n.to_csv("data/test_processed.csv",index=False)
data_test_n

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1,1.089945,-1.565186,-1.547695,-1.821155,-1.032049,1.706070,1.263248,-1.007591,-0.660957,0.182525
1,2191,2,0.544699,-1.565186,-1.414980,-1.221217,-0.994926,1.834056,1.263248,-1.007591,-0.660957,1.288280
2,2192,3,1.889640,-2.676564,-2.571492,-2.459799,-2.145743,0.426211,1.095855,-1.007591,-0.783531,-0.561346
3,2193,4,1.707891,-1.018317,-1.263307,-1.337334,-2.034373,-0.981635,-1.749836,0.944687,-1.028679,2.826284
4,2194,5,1.580667,-1.812159,-1.926879,-3.040384,-2.999575,-1.877537,-1.526645,1.522121,-1.028679,-0.310038
...,...,...,...,...,...,...,...,...,...,...,...,...
725,2915,361,1.326219,-1.441699,-1.206429,-1.163159,-1.254788,1.706070,1.040057,-1.007591,-1.028679,1.187757
726,2916,362,-0.327696,-0.559653,-1.111633,-1.182511,-0.827871,-0.597677,0.203089,-0.567641,-0.783531,0.272996
727,2917,363,1.671542,-0.947753,-1.035796,-0.988983,-0.920679,1.194127,1.095855,-1.007591,-0.660957,-0.058730
728,2918,364,0.163026,-0.947753,-0.751408,-0.466456,-0.122532,1.450099,0.928461,-1.007591,-0.660957,1.710477


### Imbalanced Dataset 

In [41]:
targets.value_counts()

rainfall
1    1650
0     540
Name: count, dtype: int64

In [60]:
from imblearn.over_sampling import SMOTE


#update the value of features and targets after normalize 
features=data_train_n.drop(columns=['id','day','rainfall'])
targets=data_train_n["rainfall"]



smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(features.values, targets.values)


In [58]:
y_r = pd.Series(y_resampled,name="rainfall")
y_r.value_counts()

rainfall
1    1650
0    1650
Name: count, dtype: int64

In [96]:
data_train_VF= pd.DataFrame(X_resampled,columns=features.columns)
data_train_VF['rainfall']= y_resampled
data_train_VF['id']=np.arange(len(data_train_VF))+1
data_train_VF['day']=np.arange(len(data_train_VF)) % 366 +1 

#reorde columns and save it 
columns =['id','day'] + list(features.columns) +  ["rainfall"]
data_train_VF = data_train_VF[columns]
data_train_VF.to_csv("data/train_processed.csv",index=False)

In [97]:
data_train_VF

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,1,1,0.671702,-0.913809,-0.642199,-0.448815,-0.199457,0.636434,0.681269,-0.729397,-0.560901,-0.465291,1
1,2,2,1.043116,-1.798289,-1.350846,-1.259418,-0.956001,1.662224,0.847728,-1.032804,-0.685925,0.009629,1
2,3,3,1.856688,-1.232222,-1.504067,-1.496667,-2.109731,-0.902250,-1.593680,1.256536,-0.435876,-0.374349,1
3,4,4,-0.035752,-1.462187,-1.178472,-1.041939,-0.691210,1.662224,1.069675,-1.032804,-0.560901,1.393971,1
4,5,5,1.449902,-0.896120,-1.063556,-1.378043,-2.052990,-3.851394,-1.704654,-0.039837,-0.810950,0.302665,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,3296,2,0.896361,-0.173770,-0.220368,-0.056342,-0.682610,-0.089462,-0.909499,-0.652118,-0.748952,0.660752,0
3296,3297,3,1.614381,-1.623131,-1.710886,-1.807024,-1.973620,-1.094424,-1.688022,1.275851,-1.061000,-0.934382,0
3297,3298,4,-0.910964,0.912511,0.916527,0.891618,0.909954,0.101534,-0.315840,1.454347,0.241032,-0.326925,0
3298,3299,5,-0.952384,0.979646,1.036035,0.931877,0.857713,-0.849226,-1.584084,1.919338,1.586145,-1.007147,0


### Dimension Reduction 

> Apply Principal Component  Analysis

In [118]:
from sklearn.decomposition import PCA

train=pd.read_csv("data/train_processed.csv",dtype=np.float32)
features=train.drop(columns=['id','day','rainfall'])
targets=train['rainfall']

pca = PCA(n_components=5,svd_solver="full") # from 10 to 5 

feat_pca=pca.fit_transform(features)
pca_df = pd.DataFrame(feat_pca, columns=["PC1", "PC2","PC3","PC4","PC5"])
pca_df["rainfall"] = targets.values

pca_df.to_csv("data/train_pca.csv",index=False)

In [117]:
pca_df

,PC1,PC2,PC3,PC4,PC5,rainfall
0,-1.681271,1.401096,-0.815937,-0.048957,0.113072,1.0
1,-3.328763,1.886631,-0.655738,-0.097774,1.136976,1.0
2,-3.002949,-2.503245,-0.580183,0.480613,0.891330,1.0
3,-2.755050,2.368489,0.803139,-0.369434,0.995535,1.0
4,-2.930984,-3.290252,0.346370,1.098036,-1.907488,0.0
...,...,...,...,...,...,...
3295,-1.315982,0.022391,0.418847,-0.649037,-0.113708,0.0
3296,-3.310956,-2.742178,-1.245059,0.196968,0.695523,0.0
3297,2.140737,0.123733,-0.013056,-0.527242,0.164404,0.0
3298,3.061349,-1.349540,-0.194924,0.679204,0.526757,0.0


### Model 